# Bisection method

In [1]:
# Import modules
import math
import numpy as np

# Initial guess
theta1 = 60
theta2 = 90

# Assign variables
mu = 0.75   # friction coefficient
weight = 25 # Weight of the block in kN
force = 17.5 # kN

theta = 0  # angle in degrees

# Set a tolerance
tolerance = 1e-5

# Iterate to a maximum of 1000 iterations
max_iterations = 1000

iterations = 0

for i in np.arange(max_iterations):

    # Compute forces for theta1 and theta2
    delta1 = force - (mu * weight) / (math.cos(math.radians(theta1)) +
                                       mu * math.sin(math.radians(theta1)))
    
    delta2 = force - (mu * weight) / (math.cos(math.radians(theta2)) +
                                       mu * math.sin(math.radians(theta2)))
    
    # Compute the mid-value of theta
    theta = (theta1 + theta2)/2
    
    # Calculate the difference delta for the mid-theta   
    delta = force - (mu * weight) / (math.cos(math.radians(theta)) +
                                       mu * math.sin(math.radians(theta)))
    
    if((delta * delta1) > 0):
        theta1 = theta
    else:
        theta2 =  theta
    
    # Final values at the end of iterations
    iterations = i
    
    if (abs(delta) <= tolerance):
        print(delta)
        break

else: #No break
    print("Solution did not converge!")
    
final_force = (mu * weight) / (math.cos(math.radians(theta)) +
                                       mu * math.sin(math.radians(theta)))

print('After', iterations, 'iterations the angle is {:.6f} deg, \
which gives a force of {:.10f} kN at a tolerance of {:.2E}'.format(theta, final_force, tolerance))

-5.229462054501255e-07
After 14 iterations the angle is 67.872620 deg, which gives a force of 17.5000005229 kN at a tolerance of 1.00E-05


# Solving a non-linear equation by Newton Raphson method

Let's go back to the sliding block problem.

The force equlibrium equation will give the following solution for F. It is a nonlinear function of $\theta$

$$
F = \frac{\mu W}{\cos \theta + \mu \sin \theta} = \frac{\mu mg}{\cos \theta + \mu \sin \theta}
$$

Given F=17.5kN, W=25kN and 𝜇 =0.75, what is 𝜃 ?

Let's make a new function 

$$
G(\theta) = \frac{\mu mg}{\cos \theta + \mu \sin \theta} - 17.5
$$

The goal is to find $\theta$ that makes G($\theta$)=0. We will use the Newton Raphson method to find the solution numerically.

# Newton Raphson Method

The general form of Taylor series is given by the following equation.

$$
P(x) = f(a) + \frac{df(a)}{dx}\frac{(x-a)^1}{1!} + \frac{d^2f(a)}{dx^2}\frac{(x-a)^2}{2!} + \cdots
$$

If x = $\theta_{n+1}$ = $\theta_n$ + $\epsilon_n$ and a = $\theta_n$, then G($\theta$)=0 can be approximately as follows.

$$
G(\theta_{n+1}) = G(\theta_n + \epsilon_n) = 0 = G(\theta_n) + \frac{dG(\theta_n)}{d\theta}\frac{(\epsilon_n)^1}{1!} + \frac{d^2G(\theta_n)}{d\theta^2}\frac{(\epsilon_n)^2}{2!} + \cdots
$$ 

By only using the first derivative term,

$$
G(\theta_n + \epsilon_n) = 0 \approx G(\theta_n)+ \frac{dG(\theta_n)}{d\theta}\epsilon_n
$$

$$
\epsilon_n = - \frac{G(\theta_n)}{dG(\theta_n)/d\theta}
$$

Hence 

$$
\theta_{n+1} = \theta_n + \epsilon_n = \theta_n - \frac{G(\theta_n)}{dG(\theta_n)/d\theta}
$$

If the gradient dG($\theta$)/dx is known, then the above equation can be used iteratively from an initial guess $\theta_0$ to find $\theta$ that satisfies G($\theta$)=0. The iteration can stop when $|\epsilon_n|$ becomes smaller than the predefined tolarance. 

The first derivate of G($\theta$) is given as follpws.

$$
dG(\theta)/d(\theta) = \frac{\mu mg (\sin \theta - \mu \cos \theta)}{(\cos \theta + \mu \sin \theta)^2}
$$

Taylor's series is used for the cosin and sin functions.

$$
\cos(\theta) \approx cos(0) - cos(0)\frac{\theta^2}{2!} + cos(0)\frac{\theta^4}{4!} + \cdots
$$

$$
\sin(\theta) \approx cos(0)\frac{\theta}{1!} - cos(0)\frac{\theta^3}{3!} + \cdots
$$

In [ ]:
import math

# The approximate cosine function
def approx_cosx(x):
        cos_approx = 0
        for i in range(n0):
            coef = (-1)**i
            num = x**(2*i)
            denom = math.factorial(2*i)
            cos_approx += ( coef ) * ( (num)/(denom) )
        return cos_approx

# The approximate sine function

def approx_sinx(x):
        sin_approx = 0
        for i in range(n0):
            coef = (-1)**(i+1)
            num = x**(2*i+1)
            denom = math.factorial(2*i+1)
            sin_approx -= ( coef ) * ( (num)/(denom) )
        return sin_approx

$$F' = \frac{\mu mg (\sin \theta - \mu \cos \theta)}{(\cos \theta + \mu \sin \theta)^2}$$

In [17]:
import sympy as sym
theta = sym.Symbol('theta')
mu = sym.Symbol('mu')
m = sym.Symbol('m')
g = sym.Symbol('g')
f =  mu * m * g / (sym.cos(theta) + mu * sym.sin(theta))
sym.diff(f, theta)

g*m*mu*(-mu*cos(theta) + sin(theta))/(mu*sin(theta) + cos(theta))**2

In [3]:
from scipy.misc import derivative
import math

# Initial values assumed 
x0 = math.radians(60)
mu = 0.75
Weight = 25

# Define a function
def fn(angle):
    theta = angle
    return mu*Weight/(math.cos(theta)+mu*math.sin(theta))-17.5

# Define dG/dtheta function
def dfn(angle):
    theta = angle
    #return derivative(fn, angle, dx=1e-6, n=1)
    return mu*Weight*(math.sin(theta)-mu*math.cos(theta))/(math.cos(theta)+ mu*math.sin(theta))**2

 
# Function to find the root 
def newton_raphson(x, max_iterations=10000, tol=1e-5): 
    for i in range(max_iterations):
      h = fn(x) / dfn(x) 
      # x(i+1) = x(i) - f(x) / f'(x) 
      x = x - h
      if abs(h) < tol:    
        return x, h, i
    else: #nobreak
      print("Solution did not converge")


# Force
angle, error, iterations = newton_raphson(x0)
final_force = fn(angle)+17.5

# Relative error
relative_error = abs(error / final_force) * 100

print('After', iterations, 'iterations the angle is {:.6f} deg, \n \
        which gives a force of {:.10f} kN, \n \
        and an absolute error of {:.10E}, \n \
        and a relative error of {:.10E} %.'.format(math.degrees(angle), final_force , error, relative_error))

After 3 iterations the angle is 67.872617 deg, 
         which gives a force of 17.5000000002 kN, 
         and an absolute error of 3.3170883848E-06, 
         and a relative error of 1.8954790770E-05 %.
